<a href="https://colab.research.google.com/github/sttesta3/7506R-1C2024-GRUPO08/blob/santiago/Regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# General libs
import pandas as pd
import numpy as np

#Visualizaciones
import seaborn as sns
import matplotlib.pyplot as plt

#Modelos
from sklearn.neighbors import KNeighborsRegressor

In [52]:
# Ruta donde van a estar todos los datasets
ruta = "https://raw.githubusercontent.com/sttesta3/7506R-1C2024-GRUPO08/santiago/datasets/"
x_train = pd.read_csv(ruta + "x_columnas_entrenamiento.csv")
y_train = pd.read_csv(ruta + "y_precios_entrenamiento.csv")

x_test = pd.read_csv(ruta + "x_columnas_test.csv")
y_test = pd.read_csv(ruta + "y_precios_test.csv")

# Creamos dataset de TEST y limpiamos
test = x_test.join(y_test)
test_knn = test.select_dtypes(include='float64')

test_knn.info()
test_knn.dropna(inplace=True)
test_knn.info()

x_test_knn=pd.DataFrame(test_knn, columns=["latitud","longitud","property_rooms","property_surface_total","property_surface_covered"])
y_test_knn=test_knn.property_price

# Creamos dataset de TEST y limpiamos
x_train_knn = x_train.select_dtypes(include='float64')
y_train_knn = y_train.select_dtypes(include='float64')
train_knn = x_train_knn.join(y_train_knn)

train_knn.info()
train_knn.dropna(inplace=True)
train_knn.info()

y_train_knn=train_knn.property_price
x_train_knn=pd.DataFrame(train_knn, columns=["latitud","longitud","property_rooms","property_surface_total","property_surface_covered"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19099 entries, 0 to 19098
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   latitud                   18291 non-null  float64
 1   longitud                  18291 non-null  float64
 2   property_rooms            18879 non-null  float64
 3   property_bedrooms         16862 non-null  float64
 4   property_surface_total    18122 non-null  float64
 5   property_surface_covered  18402 non-null  float64
 6   property_price            18839 non-null  float64
dtypes: float64(7)
memory usage: 1.0 MB
<class 'pandas.core.frame.DataFrame'>
Index: 14558 entries, 0 to 19097
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   latitud                   14558 non-null  float64
 1   longitud                  14558 non-null  float64
 2   property_rooms            145

In [54]:
# Creo un modelo KNN para regresion y lo ajusto
knn_regressor = KNeighborsRegressor().fit(x_train_knn, y_train_knn)
y_pred=knn_regressor.predict(x_test_knn)
y_pred

performance = pd.DataFrame({'Valor Real': y_test_knn,
                            'Prediccion': y_pred,
                            'Error': y_test_knn - y_pred})
# View
performance.head()

from sklearn import metrics
mse = metrics.mean_squared_error(
        y_true  = y_test_knn,
        y_pred  = y_pred,
        squared = True
       )

rmse = metrics.mean_squared_error(
        y_true  = y_test_knn,
        y_pred  = y_pred,
        squared = False
       )

print(f"El error (mse) de test es: {mse}")
print(f"El error (rmse) de test es: {rmse}")

El error (mse) de test es: 22797010116.966652
El error (rmse) de test es: 150986.78788876414
